## WHO GHO indicators

Currently trying to deal with idempotency issue when trying to change country codes into names and rerunning scripts

In [37]:
# import libraries (installed in epi-core environment)
import pandas as pd
import requests
import sqlalchemy
from sqlalchemy import create_engine, text, URL
import json
from pathlib import Path
from dotenv import load_dotenv
import os

In [38]:
# check for .env

env_path = Path.cwd().parent / "env" / ".env"  # up one folder

if not os.path.exists(env_path):
    sys.exit(f"Missing {env_path} file. Please create one with your DB credentials.")

load_dotenv(dotenv_path=env_path) 

PG_USER = os.getenv("PGUSER")
PG_PASSWORD = os.getenv("PGPASSWORD")
PG_HOST = os.getenv("PGHOST")
PG_PORT = os.getenv("PGPORT")
PG_DATABASE = os.getenv("PGDATABASE")

# Validate
missing_vars = [var for var, val in {
    "PG_USER": PG_USER,
    "PG_PASSWORD": PG_PASSWORD,
    "PG_HOST": PG_HOST,
    "PG_PORT": PG_PORT,
    "PG_DATABASE": PG_DATABASE
}.items() if val is None]

if missing_vars:
    print(f"Missing required environment variables: {', '.join(missing_vars)} "
             f"\nMake sure they are defined in {env_path}")

In [39]:
# configuration

pg_url = URL.create(
    "postgresql+psycopg",
    username=os.getenv("PGUSER"),
    password=os.getenv("PGPASSWORD"),
    host=os.getenv("PGHOST", "localhost"),
    port=int(os.getenv("PGPORT", 5432)),
    database=os.getenv("PGDATABASE"),
)

INDICATORS = ["WHOSIS_000001", "WHOSIS_000002", "MDG_0000000026", "u5mr", "MDG_0000000020"]
BASE_URL = "https://ghoapi.azureedge.net/api"  # WHO GHO OData endpoint

In [40]:
# connect to db
engine = create_engine(pg_url, pool_pre_ping=True)

with engine.connect() as conn:
    # Drop existing tables if you want a clean run
    conn.execute(text("""
    DO $$ DECLARE r RECORD;
    BEGIN
        FOR r IN (SELECT tablename FROM pg_tables WHERE schemaname = 'public') LOOP
            EXECUTE 'DROP TABLE IF EXISTS public.' || quote_ident(r.tablename) || ' CASCADE';
        END LOOP;
    END $$;
    """))
    conn.commit()

In [41]:
# create schema by running sql file

def run_sql_file(engine, path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"SQL file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        sql_text = f.read()
    # exec_driver_sql allows multiple statements separated by semicolons
    with engine.begin() as conn:      # begins a transaction
        conn.exec_driver_sql(sql_text)


try:
    run_sql_file(engine, os.path.join("..", "scripts", "schema.sql") if os.path.basename(os.getcwd())=="notebooks" else "scripts/schema.sql")
    print("Schema created/verified.")
except Exception as e:
    print("Failed to apply SQL files:", e)
    raise

Schema created/verified.


In [42]:
# extract

# all_records = []

# for ind in INDICATORS:
#     print(f"Fetching {ind}...")
#     url = f"{BASE_URL}/{ind}"
#     resp = requests.get(url)
#     if resp.status_code != 200:
#         print(f"Failed to fetch {ind}: {resp.status_code}")
#         continue
#     data = resp.json().get("value", [])
#     for row in data:
#         all_records.append({
#             "indicator_id": ind,
#             "indicator_name": row.get("IndicatorName"),
#             "country_code": row.get("SpatialDim"),
#             "country_name": row.get("SpatialDimType"),  # WHO uses this as country name for some datasets
#             "region": row.get("ParentLocation"),
#             "year": int(row.get("TimeDim")) if row.get("TimeDim") else None,
#             "value": float(row.get("NumericValue")) if row.get("NumericValue") else None
#         })

# df = pd.DataFrame(all_records)

In [43]:
# country metadata for country names

print("Fetching country metadata...")
resp = requests.get(f"{BASE_URL}/Dimension/COUNTRY/DimensionValues")
if resp.status_code != 200:
    raise RuntimeError("Failed to fetch country dimension data")

country_data = pd.DataFrame(resp.json().get("value", []))
country_lookup = country_data[["Code", "Title", "ParentTitle"]].rename(
    columns={
        "Code": "country_code",
        "Title": "country_name",
        "ParentTitle": "region"
    }
)

Fetching country metadata...


In [44]:
all_records = []
for ind in INDICATORS:
    print(f"Fetching indicator: {ind}")
    url = f"{BASE_URL}/{ind}"
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Failed to fetch {ind}: {resp.status_code}")
        continue
    data = resp.json().get("value", [])
    for row in data:
        all_records.append({
            "indicator_id": ind,
            "indicator_name": row.get("IndicatorName"),
            "country_code": row.get("SpatialDim"),
            "year": int(row.get("TimeDim")) if row.get("TimeDim") else None,
            "value": float(row.get("NumericValue")) if row.get("NumericValue") else None
        })

df = pd.DataFrame(all_records)

Fetching indicator: WHOSIS_000001
Fetching indicator: WHOSIS_000002
Fetching indicator: MDG_0000000026
Fetching indicator: u5mr
Fetching indicator: MDG_0000000020


In [45]:
# transform
# Join with country lookup to enrich names/regions
df = df.merge(country_lookup, on="country_code", how="left")

dim_indicator = df[['indicator_id', 'indicator_name']].drop_duplicates()
dim_country = df[['country_code', 'country_name', 'region']].drop_duplicates()
fact_measure = df[['indicator_id', 'country_code', 'year', 'value']]

# # Clean up country_name (SpatialDimType is often 'COUNTRY', so we'll fetch proper country names from WHO "Dimension" endpoint)
# # For now, we'll just map code -> itself as placeholder
# df['country_name'] = df['country_code']

# # Split into dimension + fact tables
# dim_indicator = df[['indicator_id', 'indicator_name']].drop_duplicates()
# dim_country = df[['country_code', 'country_name', 'region']].drop_duplicates()
# fact_measure = df[['indicator_id', 'country_code', 'year', 'value']]


In [47]:
# load old
# dim_indicator.to_sql('dim_indicator', engine, if_exists='append', index=False)
# dim_country.to_sql('dim_country', engine, if_exists='append', index=False)
# fact_measure.to_sql('fact_measure', engine, if_exists='append', index=False)

# print("ETL complete. Data loaded into epidemiology database.")
# print(f"Indicators: {len(dim_indicator)}, Countries: {len(dim_country)}, Records: {len(fact_measure)}")

In [ ]:
# new load using postgreSQL upserts (SQLAlchemy)
# UPSERTs dim_indicator and dim_country (update names/region on conflict)
# UPSERTs or DO NOTHING for fact_measure on (indicator_id, country_code, year)
# Handles NaN → None, chunks large inserts, and logs runs to etl_run_log

In [48]:
import math
from sqlalchemy import text
from sqlalchemy.dialects.postgresql import insert as pg_insert
from sqlalchemy import Table, MetaData

In [49]:
# Choose behavior for fact table:
#   "update"  -> overwrite value on conflict
#   "nothing" -> keep existing value unchanged on conflict
FACT_CONFLICT_MODE = "update"  # or "nothing"

In [51]:
# Convert NaN -> None for safe DB inserts
def _nan_to_none(df):
    return df.where(pd.notnull(df), None)

dim_indicator = _nan_to_none(dim_indicator)
dim_country   = _nan_to_none(dim_country)
fact_measure  = _nan_to_none(fact_measure)

In [52]:
meta = MetaData()
meta.reflect(bind=engine, schema="public")
t_indicator = Table("dim_indicator", meta, autoload_with=engine, schema="public")
t_country   = Table("dim_country",   meta, autoload_with=engine, schema="public")
t_fact      = Table("fact_measure",  meta, autoload_with=engine, schema="public")
t_log       = Table("etl_run_log",   meta, autoload_with=engine, schema="public")

In [53]:
def chunk_iterable(records, size=5000):
    total = len(records)
    for i in range(0, total, size):
        yield records[i:i+size]

def upsert_dimension(conn, table, rows, pk_col, update_cols):
    if not rows:
        return 0
    stmt = pg_insert(table).values(rows)
    stmt = stmt.on_conflict_do_update(
        index_elements=[table.c[pk_col]],
        set_={c: stmt.excluded[c] for c in update_cols}
    )
    result = conn.execute(stmt)
    return result.rowcount or 0

def upsert_fact(conn, rows):
    if not rows:
        return 0
    stmt = pg_insert(t_fact).values(rows)
    if FACT_CONFLICT_MODE == "update":
        # Update the metric value if source data changes
        stmt = stmt.on_conflict_do_update(
            index_elements=[t_fact.c.indicator_id, t_fact.c.country_code, t_fact.c.year],
            set_={"value": stmt.excluded.value}
        )
    else:
        # Keep existing fact; ignore duplicates
        stmt = stmt.on_conflict_do_nothing(
            index_elements=[t_fact.c.indicator_id, t_fact.c.country_code, t_fact.c.year]
        )
    result = conn.execute(stmt)
    return result.rowcount or 0

In [57]:
# Prepare records as list[dict]
ind_rows = dim_indicator.to_dict(orient="records")
cty_rows = dim_country.to_dict(orient="records")
fact_rows = fact_measure.to_dict(orient="records")

ProgrammingError: (psycopg.errors.CardinalityViolation) ON CONFLICT DO UPDATE command cannot affect row a second time
HINT:  Ensure that no rows proposed for insertion within the same command have duplicate constrained values.
[SQL: INSERT INTO public.fact_measure (indicator_id, country_code, year, value) VALUES (%(indicator_id_m0)s::VARCHAR, %(country_code_m0)s::VARCHAR, %(year_m0)s::INTEGER, %(value_m0)s), (%(indicator_id_m1)s::VARCHAR, %(country_code_m1)s::VARCHAR, %(year_m1)s::INTEGER, %(value_m1)s), (%(indicator_id_m2)s::VARCHAR, %(country_code_m2)s::VARCHAR, %(year_m2)s::INTEGER, %(value_m2)s), (%(indicator_id_m3)s::VARCHAR, %(country_code_m3)s::VARCHAR, %(year_m3)s::INTEGER, %(value_m3)s), (%(indicator_id_m4)s::VARCHAR, %(country_code_m4)s::VARCHAR, %(year_m4)s::INTEGER, %(value_m4)s), (%(indicator_id_m5)s::VARCHAR, %(country_code_m5)s::VARCHAR, %(year_m5)s::INTEGER, %(value_m5)s), (%(indicator_id_m6)s::VARCHAR, %(country_code_m6)s::VARCHAR, %(year_m6)s::INTEGER, %(value_m6)s), 